In [ ]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import getpass

from rich.progress import track

In [ ]:
class Scraper:

    def __init__(self, driver_path=rf"/home/{getpass.getuser()}/Documents/", driver_type="Firefox"):
        if driver_type=="Firefox":
            self.webdriver = driver_path + "geckodriver"
            op = webdriver.FirefoxOptions()
            op.add_argument('--headless')
            self.driver = webdriver.Firefox(executable_path=self.webdriver, options=op)
        elif driver_type=="Chrome":
            self.webdriver = driver_path + "chromedriver"
            op = webdriver.ChromeOptions()
            op.add_argument('--headless')
            self.driver = webdriver.Chrome(executable_path=self.webdriver, options=op)
        
    def etf_scrape(self):
        self.driver.get("https://www.barchart.com/options/iv-rank-percentile/etfs?orderBy=optionsImpliedVolatilityPercentile1y&orderDir=desc")

        ticker_name = [self.driver.find_element_by_xpath(f"/html/body/main/div/div[2]/div[2]/div/div[2]/div/div/div/div[6]/div/div[2]/div/div/ng-transclude/table/tbody/tr[{i}]/td[1]/div/span[2]/a").text for i in track(range(1,101), description="Downloading Ticker Name...")]
        ticker_last = [self.driver.find_element_by_xpath(f"/html/body/main/div/div[2]/div[2]/div/div[2]/div/div/div/div[6]/div/div[2]/div/div/ng-transclude/table/tbody/tr[{i}]/td[3]/div/span/span/span").text for i in track(range(1,101), description="Downloading Last Price...")]
        ticker_optvol = [self.driver.find_element_by_xpath(f"/html/body/main/div/div[2]/div[2]/div/div[2]/div/div/div/div[6]/div/div[2]/div/div/ng-transclude/table/tbody/tr[{i}]/td[6]/div/span/span/span").text for i in track(range(1,101), description="Downloading Option Vol...")]
        ticker_impr = [self.driver.find_element_by_xpath(f"/html/body/main/div/div[2]/div[2]/div/div[2]/div/div/div/div[6]/div/div[2]/div/div/ng-transclude/table/tbody/tr[{i}]/td[8]/div/span/span/span").text for i in track(range(1,101), description="Downloading IV Rank...")]
        ticker_impp = [self.driver.find_element_by_xpath(f"/html/body/main/div/div[2]/div[2]/div/div[2]/div/div/div/div[6]/div/div[2]/div/div/ng-transclude/table/tbody/tr[{i}]/td[9]/div/span/span/span").text for i in track(range(1,101), description="Downloading IV Percentile...")]

        return list(zip(ticker_name, ticker_last, ticker_optvol, ticker_impr, ticker_impp))

In [ ]:
scrap = Scraper()

scrap.etf_scrape()

In [ ]:
from rich.console import Console
from rich.table import Table

class Renderer:
    def __init__(self):
        self.console = Console()
    
    def last_color(self, text):
        if float(text) < 30:
            return f"[bright_green]{str(text)}[/]"
        elif float(text) > 30 and float(text) < 50:
            return f"[orange1]{str(text)}[/]"
        else:
            return f"[bright_red]{str(text)}[/]"

    def screener_table(self, data, table_type="ETF"):
        table = Table(title=f"Screened {table_type}")
        col_name = ["Ticker", "Last", "Volume", "IV Rank", "IV Perc"]

        for i in range(0,5):
            table.add_column(col_name[i])

        for i in range(0,11):
            table.add_row(data[i][0], self.last_color(data[i][1]),data[i][2],data[i][3],data[i][4])

        self.console.print(table)

In [5]:
import uuid
import datetime

class Trade:

    def __init__(self, open_date: str, ticker: str, strike_price : float, trade_price: float, exp_date: str, roll_count = 0) -> None:
        self.open_date = datetime.datetime.strptime(open_date, "%d %b %Y")
        self.ticker = ticker
        self.strike_price = strike_price
        self.trade_price = trade_price
        self.exp_date = datetime.datetime.strptime(exp_date, "%d %b %Y")
        self.roll_count = roll_count
        self.pnl = 0
        self.dte = (datetime.datetime.now() - self.exp_date).days
        self.breakeven = self.strike_price - self.trade_price

    def __str__(self) -> str:
        return str([self.open_date.strftime("%d %b %Y"), str(self.ticker), float(self.strike_price), float(self.trade_price),
        self.exp_date.strftime("%d %b %Y"), int(self.roll_count)])

    def roll_trade(self, roll_price : float, exp_date: str, strike_price = None) -> None:
        if strike_price is None:
            self.strike_price = self.strike_price
        else:
            self.strike_price = strike_price
        self.roll_count += 1
        self.trade_price += roll_price
        self.exp_date = datetime.datetime.strptime(exp_date, "%d %b %Y")

    def close_trade(self, close_price: float) -> float:
        self.roll_count += 1 
        self.pnl = (self.trade_price - close_price)*100 - 0.70*(self.roll_count*2)

        return self.pnl

class Account:

    def __init__(self, deposits: float, curr_value: float, open_trades: dict) -> None:
        self.base = deposits
        self.value = curr_value
        self.open_trades = open_trades
        pass

    def add_value(self, top_up):
        self.base += top_up

    def new_trade(self, open_date: str, ticker: str, strike_price : float, trade_price: float, exp_date: str) -> None:
        new_trade = Trade(open_date, ticker, strike_price, trade_price, exp_date) 
        self.open_trades[ticker + " " + str(open_date)] = new_trade

    def roll_trade(self, trade_name: str, roll_price: float, exp_date: str, strike_price = None) -> None:
        trade = self.open_trades.get(trade_name)
        trade.roll_trade(roll_price, exp_date, strike_price)
        pass

    def close_trade(self, trade_name, close_price: float) -> None:
        trade = self.open_trades.get(trade_name)
        pnl = trade.close_trade(close_price)
        self.value += pnl
        del self.open_trades[trade_name]

    def render_table(self):

        open_date = [self.open_trades[key].open_date.strftime("%d %b %Y") for key in self.open_trades]
        ticker_name = [str(self.open_trades[key].ticker) for key in self.open_trades]
        trd_price = [str(self.open_trades[key].trade_price) for key in self.open_trades] 
        stock_be = [str(self.open_trades[key].breakeven) for key in self.open_trades]
        dte = [str(abs(self.open_trades[key].dte)) for key in self.open_trades]

        return list(zip(open_date, ticker_name, trd_price, stock_be, dte))

    def get_trade_attr(self, trade_name: str) -> list:
        return self.open_trades.get(trade_name).ticker

    @staticmethod
    def save_acc(account, filename):

        data = {}
        data["trades"] = account.open_trades
        data["balances"] = {'base_val' : account.base, "curr_val" : account.value}

        with open(filename+".pickle", 'wb') as handle:
            pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

    @staticmethod
    def load_acc(filename):
        with open(filename, "rb") as handle:
            data = pickle.load(handle)

        trades = data["trades"]
        bal = data["balances"]

        account = Account(bal["base_val"], bal["curr_val"], trades)

        return account

In [6]:
zach_account = Account(1000, 1000, {})

zach_account.new_trade("7 Sep 2021", "AAPL", 100, 0.50, "25 Oct 2021")
zach_account.new_trade("11 Oct 2021", "F", 23, 0.40, "25 Oct 2021")

In [ ]:
import easygui as g 

class Prompts():

    def __init__(self) -> None:
        pass

    @staticmethod
    def on_load() -> Account:
        if g.buttonbox(msg="Do you have an account?", choices=["Yes", "No"]) == "Yes":
            account = Account.load_acc(g.fileopenbox(msg="Please select your account file"))

        else:
            account_info = g.multenterbox(msg="Please fill in the basic information for your account",
            fields=["Deposit Amount", "Current Value"])
            if account_info is None:
                return Prompts.on_load()
            else:
                account = Account(account_info[0], account_info[1], {})

        return account

    @staticmethod
    def open_trade(account) -> None:
        fieldValues = g.multenterbox(msg="Enter trade information", title="Trade Information", 
        fields=["Open Date", "Ticker", "Strike", "Trade Price", "Expiration Date"])

        account.new_trade(fieldValues[0], fieldValues[1], float(fieldValues[2]), float(fieldValues[3]), fieldValues[4])

    @staticmethod
    def edit_trade(account) -> None:
        trdname = g.choicebox("Pick an item", "ITEM PICKER", choices=list(account.open_trades.keys()))

        if trdname not in account.open_trades:
            g.msgbox(msg="Trade not found. Please try again.")
        else:
            choiceValues = g.buttonbox(msg="Close or Roll Trade?", choices=["Close", "Roll"])
            if choiceValues == "Roll":
                fieldValues = g.multenterbox(msg="Enter updated trade information", title="Trade Editor",
                fields=["Roll Price", "Expiration Date", "Strike (Optional)"], values=[0,0,""])
                if (fieldValues is None):
                    pass
                else:
                    account.roll_trade(trdname, float(fieldValues[0]), str(fieldValues[1]), 
                    float(fieldValues[2]) if fieldValues[2] != "" else fieldValues[2])
            elif choiceValues == "Close":
                fieldValues = g.multenterbox(msg="Enter closing trade information", title="Closing Trade",
                fields=["Close Price"], values=[0])
                if (fieldValues is None):
                    pass
                else:
                    account.close_trade(trdname, float(fieldValues[0]))

In [ ]:
Prompts.edit_trade(zach_account)

In [ ]:
from yahoo_fin import stock_info as si

si.get_futures().loc[0:2], 

In [ ]:
data = list(zip(si.get_day_losers(count=5).loc[:,"Symbol"], si.get_day_losers(count=5).loc[:, "% Change"],
si.get_day_gainers(count=5).loc[:,"Symbol"], si.get_day_gainers(count=5).loc[:, "% Change"]))

In [ ]:
from rich.table import Table
from rich import print

table = Table(title=f"Market")
col_name = ["Ticker", "Last", "% Change"]

for i in range(len(col_name)):
    table.add_column(col_name[i])

for i in range(len(data)):
    table.add_row(data[i][0], str(data[i][1]), data[i][2], str(data[i][3]))

print(table)

In [ ]:
from rich.progress import track

load_list = [si.get_futures, si.get_market_status, si.get_day_gainers, si.get_day_losers]

data_list = []

for i in track(range(len(load_list)),description="Loading data"):
    data_list.append(load_list[i]())

futs = data_list[0].loc[0:3]
status = data_list[1]
tg = data_list[2].loc[0:4]
tl = data_list[3].loc[0:4]

In [ ]:
status

In [ ]:
round(si.get_live_price("VXX"),2)

In [ ]:
from rich import print

change = "3.5%"

inplant = f""

print("SPY " + color(change))

In [ ]:
def color(text):

    return f"[chartreuse2]{text}" if float(change.removesuffix("%")) > 0 else f"[deep_pink4]{change}" 

In [9]:
import easygui as g

In [3]:
import warnings

def f():
    print("before")
    warnings.warn("You are warned!")
    print("after")

warnings.filterwarnings("ignore")

In [18]:
input = g.enterbox()
type(input)

input == ""

True

In [1]:
import easygui as g

In [24]:
def random():
    while True:
        if g.enterbox() == "yes":
           return False

random()

False

In [7]:
g.ynbox(msg="Account Management", choices=["Change Deposit Amount", "Save Account"])

True

In [3]:
import pickle

with open("toight.pickle", "rb") as file:
    acc = pickle.load(file)

acc

{'trades': {},
 'balances': {'base_val': 1200.0, 'curr_val': 1657.1999999999998}}

In [4]:
import os 

os.path.getmtime("toight.pickle")

1634784519.693679